# Tâche #1 : Classification d'incidents avec des modèles *Transformers*

On reprend, comme au premier travail, le thème de la classification de descriptions d’incidents. Cependant, la définition des classes est différente pour ce travail et de nouveaux jeux de données ont été produits. Le corpus de textes contient 3 partitions : 
-	Un fichier d’entraînement -  *data/incidents_train.json*
-	Un fichier de validation -  *data/incidents_dev.json*
-	Un fichier de test - *data/incidents_test.json*
 
Utilisez la librairie *HuggingFace* pour accomplir cette tâche. On vous demande plus spécifiquement d’utiliser 2 modèles: le modèle *bert-base-uncased* et un autre modèle de votre choix. 

Les consignes pour cette tâche sont: 
- Nom du notebook : *t1_classification.ipynb* (ce notebook)
- Tokenisation : Celle fournie par les tokeniseurs accompagnant les modèles transformers. 
- Plongements de mots : Ceux du modèle transformer. 
- Normalisation : Lettre en minuscule pour Bert (rien à faire, le tokenizer s’en occupe). Aucune contrainte pour le 2e modèle mais il est préférable de ne pas altérer le texte (sauf minuscule). 
- Choix du 2e transformer: Un modèle encodeur préentraîné pour l’anglais ou multilingue. Le modèle ne doit pas être une variante de Bert (p. ex. DistilBert). Me consulter en cas de doute.
- Entraînement : Un affinage (*fine-tuning*) du modèle, pas de préentraînement demandé (*no further pretraining*). 
- Analyse : Présentez clairement vos résultats et faites-en l’analyse. Comparez les résultats obtenus avec les 2 modèles.    

Vous pouvez ajouter au *notebook* toutes les cellules dont vous avez besoin pour votre code, vos explications ou la présentation de vos résultats. Vous pouvez également ajouter des sous-sections (par ex. des sous-sections 1.1, 1.2 etc.) si cela améliore la lisibilité.

Notes :
- Évitez les bouts de code trop longs ou trop complexes. Par exemple, il est difficile de comprendre 4-5 boucles ou conditions imbriquées. Si c'est le cas, définissez des sous-fonctions pour refactoriser et simplifier votre code. 
- Expliquez sommairement votre démarche.
- Expliquez les choix que vous faites au niveau de la programmation et des modèles (si non trivial).
- Analyser vos résultats. Indiquez ce que vous observez, si c'est bon ou non, si c'est surprenant, etc. 
- Une analyse quantitative et qualitative d'erreurs est intéressante et permet de mieux comprendre le comportement d'un modèle.

## 1. Création du jeu de données (*les 3 partitions du dataset*)

In [1]:
import json

def load_json_data(filename):
    with open(filename, 'r') as fp:
        data = json.load(fp)
    return data

 Notre collaboration avec Google Colab a été motivée par le coût élevé de l'entraînement des modèles transformers et la nécessité d'un GPU performant.

In [2]:
# from google.colab import drive
# drive.mount('/content')

In [3]:
# !pip install --upgrade torch transformers


In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
import numpy as np

/home/jonathan/Documents/ulaval/nlp/tp2/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from datasets import load_dataset

data= {"train": "incidents_train.json", "dev": "incidents_dev.json","test": "incidents_test.json"}
data = load_dataset("json", data_files=data, data_dir=r"data")

Le tokenizer et les embeddings de bert-base-uncased sont utilisés ici .

In [6]:
from transformers import AutoTokenizer

bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def bert_preprocess_function(examples):
    return bert_tokenizer(examples["text"], truncation=True)

tokenized_dataset = data.map(bert_preprocess_function, batched=True)
print(tokenized_dataset["train"][0])

Map: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 531/531 [00:00<00:00, 9010.19 examples/s]

{'text': ' At approximately 8:50 a.m. on October 29  1997  Employee #1 was painting a  single story house at 2657 7th Ave  Sacramento  CA. He was caulking around the  peak of the roof line on the west side of the house  20 ft above the ground.  He was working off of a 24 ft aluminum extension ladder so that his feet were  approximately 12 to 13 feet above the ground. Employee #1 fell and suffered a  concussion and two dislocated discs in his lower back and was hospitalized.  The ladder was not secured to prevent movement.                                 ', 'label': '5', 'input_ids': [101, 2012, 3155, 1022, 1024, 2753, 1037, 1012, 1049, 1012, 2006, 2255, 2756, 2722, 7904, 1001, 1015, 2001, 4169, 1037, 2309, 2466, 2160, 2012, 20549, 2581, 5504, 13642, 11932, 6187, 1012, 2002, 2001, 6187, 5313, 6834, 2105, 1996, 4672, 1997, 1996, 4412, 2240, 2006, 1996, 2225, 2217, 1997, 1996, 2160, 2322, 3027, 2682, 1996, 2598, 1012, 2002, 2001, 2551, 2125, 1997, 1037, 2484, 3027, 13061, 5331, 10535, 206

Les labels doivent être convertis en entiers pour l'entraînement, sinon le format str pose un problème.

In [7]:
for partition in tokenized_dataset.keys():
    if 'label' in tokenized_dataset[partition].column_names:
        labels = tokenized_dataset[partition]['label']
        tokenized_dataset[partition] = tokenized_dataset[partition].remove_columns("label")
        tokenized_dataset[partition] = tokenized_dataset[partition].add_column("label", [int(value) for value in labels])


Comme pour les réseaux récurrents, il est nécessaire de faire un padding des séquences pour garantir une taille de tenseur équivalente pour chaque entrée.

In [8]:
from transformers import DataCollatorWithPadding

dataset_collator = DataCollatorWithPadding(tokenizer=bert_tokenizer)

## 2. Création des 2 modèles

### 2.1 Modèle BERT


Les différentes fonctions qui seront utilisées pour évaluer notre modèle sont implantées :

In [9]:
import evaluate
accuracy_metric = evaluate.load("accuracy", force_download=True)


In [10]:
def compute_metrics(eval_predictions):
    preds, true_labels = eval_predictions
    predicted_label = np.argmax(preds, axis=1)
    return accuracy_metric.compute(predictions=predicted_label, references=true_labels)

Ici les classes sont converties en identifiants :

In [11]:
# Initialisation des dictionnaires de correspondance label <-> id
nb_classes = len(np.unique(data['train']['label']))
id2label = {i: str(i) for i in range(nb_classes)}
label2id = {str(i): i for i in range(nb_classes)}


Utilisons le GPU  pour entraîner le modèle.

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device selected: {device}")



Device selected: cuda


In [13]:
from transformers import AutoModelForSequenceClassification,TrainingArguments, Trainer

bert_uncased_model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=nb_classes,
    id2label=id2label,
    label2id=label2id
    )

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 2.2 Deuxième modèle


Un autre modèle pré-entraîné est utilisé pour l'anglais : Electra.
Le pré-entraînement d'Electra diffère de celui de Bert. Effectivement, l'entraînement préliminaire de ce modèle repose sur une approche adversariale, ce qui permet de déterminer si des mots de la phrase sont des mots réels ou s'ils ont été générés par un modèle génératif. Il y a également une différence entre Electra et Bert car le modèle ne fait pas de NSP pour son pré-entraînement.

Les mêmes fonctions et données sont conservées, le tokenizer et le modèle sont simplement remplacés par Electra :

In [14]:
electra_tokenizer = AutoTokenizer.from_pretrained("google/electra-base-discriminator")

def electra_preprocess_function(examples):
    return electra_tokenizer(examples["text"], truncation=True)

In [15]:
electra_tokenized_data = data.map(electra_preprocess_function, batched=True)
for partition in electra_tokenized_data.keys():
    labels = electra_tokenized_data[partition]['label']
    electra_tokenized_data[partition] = electra_tokenized_data[partition].remove_columns("label")
    electra_tokenized_data[partition] = electra_tokenized_data[partition].add_column("label", [int(label) for label in labels])
data_collator = DataCollatorWithPadding(tokenizer=electra_tokenizer)


Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 531/531 [00:00<00:00, 10832.25 examples/s]


In [16]:
electra_model = AutoModelForSequenceClassification.from_pretrained(
    "google/electra-base-discriminator", 
    num_labels=nb_classes, 
    id2label=id2label, 
    label2id=label2id
).to(device)


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 3. Entraînement des 2 modèles

### 3.1 Modèle BERT


Pour l'entraînement, nous avons choisi d'utiliser 3 époques afin d'obtenir de meilleures performances, bien que cela augmente le temps d'entraînement en raison de la complexité du calcul d'attention. Les autres hyperparamètres restent ceux par défaut.

In [ ]:
training_params = TrainingArguments(
    output_dir="bert_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False
    )

trainer_instance = Trainer(
    model=bert_uncased_model,
    args=training_params,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["dev"],
    tokenizer=bert_tokenizer,
    data_collator=dataset_collator,
    compute_metrics=compute_metrics

)

trainer_instance.train()

/home/jonathan/Documents/ulaval/nlp/tp2/venv/lib/python3.12/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_101672/3465564458.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_instance = Trainer(
/home/jonathan/Documents/ulaval/nlp/tp2/venv/lib/python3.12/site-packages/torch/nn/modules/linear.py:125: UserWarning: Attempting to use hipBLASLt on an unsupported architecture! Overriding blas backend to hipblas (Triggered internally at ../aten/src/ATen/Context.cpp:296.)
  return F.linear(input, self.weight, self.bias)


Epoch,Training Loss,Validation Loss,Accuracy
1,1.079400,1.020924,0.796610
2,0.813900,0.933403,0.822976
3,0.573300,1.043175,0.832392


TrainOutput(global_step=7425, training_loss=0.8723962279040404, metrics={'train_runtime': 556.8123, 'train_samples_per_second': 13.335, 'train_steps_per_second': 13.335, 'total_flos': 555967530996570.0, 'train_loss': 0.8723962279040404, 'epoch': 3.0})

### 3.2 Deuxième modèle _ Electra


Entrainement du model Electra

In [18]:
training_params2= TrainingArguments(
    output_dir="model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False
    )

trainer_instance2 = Trainer(
    model=electra_model,
    args=training_params2,
    train_dataset=electra_tokenized_data["train"],
    eval_dataset=electra_tokenized_data["dev"],
    tokenizer=electra_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics

)

trainer_instance2.train()

/home/jonathan/Documents/ulaval/nlp/tp2/venv/lib/python3.12/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_101672/4056485202.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_instance2 = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.273400,1.064022,0.696798
2,0.986100,0.986575,0.772128
3,0.802900,1.093056,0.790960


TrainOutput(global_step=7425, training_loss=1.069408538510101, metrics={'train_runtime': 561.0239, 'train_samples_per_second': 13.235, 'train_steps_per_second': 13.235, 'total_flos': 555967530996570.0, 'train_loss': 1.069408538510101, 'epoch': 3.0})

## 4. Évaluation, analyse de résultats et comparaison des 2 modèles

### Evaluation  du model BERT




In [19]:
predictions = trainer_instance.predict(tokenized_dataset["test"])
print(predictions[2])

{'test_loss': 1.1154084205627441, 'test_accuracy': 0.775894538606403, 'test_runtime': 6.6417, 'test_samples_per_second': 79.949, 'test_steps_per_second': 79.949}


### Evaluation du model Electra

In [20]:
predictions = trainer_instance2.predict(electra_tokenized_data["test"])
print(predictions[2])

{'test_loss': 1.1527515649795532, 'test_accuracy': 0.743879472693032, 'test_runtime': 4.6396, 'test_samples_per_second': 114.45, 'test_steps_per_second': 114.45}


## analyse de résultats et comparaison des 2 modèles

1. Precision (Precision)
    BERT : Grâce à une précision de 77,59% sur le jeu de test après trois époques, BERT met en évidence sa capacité à identifier des relations complexes entre les mots dans le domaine textuel. Les résultats de cette performance  montre que les têtes self-attention permettent vraiment au modèle de comprendre  les contructions et les liens entre les mots, ce qui renforce la capacité du modèle à prédire les classes et a saisir les dépendances à long terme dans les descriptions des incidents. Cette précision est nettement supérieure à celle obtenue avec ELECTRA .

    La précision du modèle ELECTRA est de 74,39%, ce qui, même si elle est légèrement inférieure à celle de BERT, demeure compétitive. ELECTRA peut se concentrer plus rapidement sur les relations pertinentes dans les données grâce à une stratégie de pré-entraînement basée sur la discrimination (au lieu du masquage comme pour BERT), ce qui peut améliorer son efficacité .

    Conclusion sur la précision : Bien que BERT surpasse ELECTRA en termes de précision, ELECTRA reste performant et pourrait être préféré dans des cas où la vitesse et l'efficacité computationnelle sont des priorités.

2. Perte (Test Loss)
    En ce qui concerne BERT, la perte de test de 1,115 est un indicateur stable de convergence, ce qui indique que le modèle s'est correctement adapté aux données lors de l'entraînement. Cela laisse entendre que BERT peut gérer les cas complexes et réduire les erreurs de manière plus efficace, même si le nombre d'époques est relativement limité.


    Conclusion sur la perte : BERT semble mieux gérer les relations complexes et converge plus efficacement dans ce cas spécifique, tandis qu'ELECTRA, bien qu'efficace, nécessite peut-être plus d'ajustements dans le fine-tuning.

3. Vitesse d'Apprentissage et d'Inférence
    Vitesse d'apprentissage : Bien que ELECTRA soit réputé pour son efficacité en fine-tuning grâce à son approche de pré-entraînement basée sur la discrimination, dans ce cas, les résultats ne montrent pas de différence notable en termes de vitesse d'apprentissage par rapport à BERT. Cette égalité suggère que les deux modèles ont des performances d'entraînement similaires avec les paramètres choisis.

    Vitesse d'inférence : En revanche,  ELECTRA se démarque grâce à sa rapidité d'inférence. Son taux de traitement est de 114,45 échantillons par seconde, ce qui est pratiquement à la fois plus rapide que BERT (qui traite 79,95 échantillons par seconde). 